##Model suggestions from Dr. Bengfort 
1) Stick with support vector machines and logistic regression
2) Gaussian Naïve Bayes if features are normally distributed
3) Multinomial naïve bayes if not normally distributed
4) Bayesian classifiers are going to perform the worst – this can be are baseline
5) stick with support vector machines and logistic regression to preserve the linearity of the features  in an understandable way (natural binary classifiers)
6) if svc is taking too long then recommend just using Stochastic Gradient Descent or Linear SVC - should improve the performance of the support vector machine and when you are working with support vector machine, make sure your trying polynomial kernels in addition to linear kernels
7 ) Don’t know if you would get good results from Random Forest and K Near Neighbors, Gradient Boosting – might want to try them 


In [1]:
import pandas as pd
import boto3
from s3 import get_file

In [2]:
def data(data):
    df = pd.read_csv(data,sep='|')
    return df

In [19]:
s3 = boto3.resource('s3')

bucket = 's3ssp'

train_data = data(get_file(s3,bucket,download_file='Analysis_Data/master_train_playlist.csv',rename_file = 'Data/train.csv'))
test_data = data(get_file(s3,bucket,download_file='Analysis_Data/test_ssp.csv',rename_file = 'Data/test.csv'))


In [20]:
df_train = train_data.reindex(sorted(train_data.columns), axis=1)
df_test = test_data.reindex(sorted(test_data.columns), axis=1)

In [21]:
df_train = df_train.drop(columns=['playlist'#,'valence','danceability',
                                 #'energy','acousticness'
                                 ])

df_test = df_test.drop(columns=['playlist'#,'valence','danceability',
                      #'energy','acousticness'
                               ])

In [16]:
df_train.dtypes

acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                 float64
liveness            float64
loudness            float64
mode                float64
speechiness         float64
target                int64
tempo               float64
time_signature      float64
valence             float64
dtype: object

In [7]:
df_train.head()

,instrumentalness,key,liveness,loudness,mode,speechiness,target,tempo,time_signature
0,0.001185,5.0,0.1310,-10.7400,1.0,0.04040,0,120.1130,4.0
1,0.023155,3.5,0.1155,-9.5715,1.0,0.03835,0,121.6830,4.0
2,0.001006,5.0,0.1475,-10.2155,1.0,0.03755,0,110.6130,4.0
3,0.000153,5.5,0.1700,-9.9190,1.0,0.03670,0,104.0955,4.0
4,0.000338,5.5,0.1565,-9.0215,1.0,0.04775,0,116.8675,4.0


In [8]:
from sklearn.linear_model import LogisticalRegression
from sklearn.pipeline import Pipeline
from sklearn.utils.validation import check_is_fitted
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm


#RandomForestRegressor(n_estimators = 50)

X = df_train[[col for col in df_train.columns if col != 'target']]

y = df_train['target']

#enc = OneHotEncoder(handle_unknown='ignore')
#enc.fit(X)
#enc.categories_


#Support Vector Machine
estimator = svm.SVC(gamma=.001)
estimator.fit(X,y)
estimator.predict(df_test)
estimator.score(X,y)

0.9215601439736355

In [22]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X,y)
logreg.predict(df_test)
logreg.score(X,y)


/Users/adamgoldstein/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: X has 12 features per sample; expecting 8

mode = [Pipeline([
    ('std', StandardScaler()),
    ('reg'),LinearRegression(fit_intercept=False)
    ]),
    Pipeline([
    ('std', StandardScaler()),
    ('reg'),RandomForestRegressor(n_estimator=50)
    ]),
    Pipeline([
    ('std', StandardScaler()),
    ('reg'),MLPRegressor(hidden_layer_sizes=(100,100,100))
    ]),
    Pipeline([
    ('std', RobustScaler()),
    ('reg'),LinearRegression(fit_intercept=False)
    ]),
    Pipeline([
    ('std', RobustScaler()),
    ('reg'),RandomForestRegressor(n_estimator=50)
    ]),
    Pipeline([
    ('std', RobustScaler()),
    ('reg'),MLPRegressor(hidden_layer_sizes=(100,100,100))
    ]),
]


#check_is_fitted(model,['support_fit_','support_','support_vectors_'])
model.fit(X,y)
model.score(X,y)